# Table of Contents
 <p><div class="lev1 toc-item"><a href="#An-Introduction-to-Torch-Framework" data-toc-modified-id="An-Introduction-to-Torch-Framework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>An Introduction to Torch Framework</a></div><div class="lev2 toc-item"><a href="#Context" data-toc-modified-id="Context-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Context</a></div><div class="lev2 toc-item"><a href="#In-a-nutshell-:" data-toc-modified-id="In-a-nutshell-:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>In a nutshell :</a></div><div class="lev1 toc-item"><a href="#Implementations" data-toc-modified-id="Implementations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementations</a></div><div class="lev2 toc-item"><a href="#Loading-the-required-modules" data-toc-modified-id="Loading-the-required-modules-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Loading the required modules</a></div><div class="lev2 toc-item"><a href="#Initiallizing-default-parameters" data-toc-modified-id="Initiallizing-default-parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Initiallizing default parameters</a></div><div class="lev1 toc-item"><a href="#Visualizing-the-given-problem" data-toc-modified-id="Visualizing-the-given-problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Visualizing the given problem</a></div><div class="lev2 toc-item"><a href="#Linear-Models---Overview" data-toc-modified-id="Linear-Models---Overview-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Linear Models - Overview</a></div><div class="lev3 toc-item"><a href="#Linear-Models-:-definition" data-toc-modified-id="Linear-Models-:-definition-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Linear Models : definition</a></div><div class="lev3 toc-item"><a href="#Fitting-linear-model" data-toc-modified-id="Fitting-linear-model-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Fitting linear model</a></div><div class="lev2 toc-item"><a href="#Linear-Regression-for-Classification-using-Torch" data-toc-modified-id="Linear-Regression-for-Classification-using-Torch-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Linear Regression for Classification using Torch</a></div><div class="lev3 toc-item"><a href="#Batch-Gradient-Descent" data-toc-modified-id="Batch-Gradient-Descent-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Batch Gradient Descent</a></div><div class="lev2 toc-item"><a href="#Plotting-train-and-test-error" data-toc-modified-id="Plotting-train-and-test-error-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Plotting train and test error</a></div><div class="lev3 toc-item"><a href="#Stochastic-Gradient-Descent" data-toc-modified-id="Stochastic-Gradient-Descent-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Stochastic Gradient Descent</a></div><div class="lev3 toc-item"><a href="#Set-all_losses" data-toc-modified-id="Set-all_losses-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Set all_losses</a></div><div class="lev3 toc-item"><a href="#Stochastic-Gradient-Descent-implementation" data-toc-modified-id="Stochastic-Gradient-Descent-implementation-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Stochastic Gradient Descent implementation</a></div><div class="lev4 toc-item"><a href="#Using-the-StochasticGradient-trainer" data-toc-modified-id="Using-the-StochasticGradient-trainer-3.3.3.1"><span class="toc-item-num">3.3.3.1&nbsp;&nbsp;</span>Using the StochasticGradient trainer</a></div><div class="lev4 toc-item"><a href="#Adapting-the-data-to-the-trainer-persepective" data-toc-modified-id="Adapting-the-data-to-the-trainer-persepective-3.3.3.2"><span class="toc-item-num">3.3.3.2&nbsp;&nbsp;</span>Adapting the data to the trainer persepective</a></div><div class="lev4 toc-item"><a href="#Adapting-the-data-to-the-trainer-paradigm" data-toc-modified-id="Adapting-the-data-to-the-trainer-paradigm-3.3.3.3"><span class="toc-item-num">3.3.3.3&nbsp;&nbsp;</span>Adapting the data to the trainer paradigm</a></div><div class="lev1 toc-item"><a href="#Loading-up-differents-datasets" data-toc-modified-id="Loading-up-differents-datasets-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Loading up differents datasets</a></div><div class="lev1 toc-item"><a href="#Conclusion" data-toc-modified-id="Conclusion-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Conclusion</a></div>

# An Introduction to Torch Framework

## Context

* **Master DAC** Machine Learning and Deep Learning
* **Students** : Youcef Benyettou & David Panou

This notebook follows the instructions given in _"Guide LUA"_ written by **Prof. Denoyer** and **Prof. Baskiotis** at UPMC.
Its intent is to allow the newcomers to get into LUA parametric models.

## In a nutshell : 
1. A loss can :
    * Can calculate its value by comparing the output for a given input 
    * Can compute its derivative for a given entry data point
2. A module can :
    * Can compute it's output for a given input (using its forward method)
    * Can update its gradient using its entry value and is delta (i.e the derivative of its error for a given output which is transmitted by the following module  loss)
    * Can compute its delta (the derivative of its error compared to ..)


# Implementations

## Loading the required modules

In [4]:
require 'nn'
require 'gnuplot'
require 'tools'
Plot = require 'itorch.Plot'

## Initiallizing default parameters

In [5]:
-- 1: Create a artificial dataset
DIMENSION=2 -- Entry dataset features dimensions
n_points=1000 -- Number of samples
  
-- We create two gaussians
mean_positive=torch.Tensor(DIMENSION):fill(1); 
var_positive=1.0

mean_negative=torch.Tensor(DIMENSION):fill(-1); 
var_negative=1.0

xs=torch.Tensor(n_points,DIMENSION)
ys=torch.Tensor(n_points,1)
for i=1,n_points/2 do  xs[i]:copy(torch.randn(DIMENSION)*var_positive+mean_positive); ys[i][1]=1 end
for i=n_points/2+1,n_points do xs[i]:copy(torch.randn(DIMENSION)*var_negative+mean_negative); ys[i][1]=-1 end

In [6]:
cat_1 = {}
cat_1["x"] = {}
cat_1["y"] = {}

cat_2 = {}
cat_2["x"] = {}
cat_2["y"] = {}

l = 1
j = 1
for i=1,xs:size()[1] do
    if ys[i][1]==1 then
        cat_1["x"][l]= xs[i][1]
        cat_1["y"][l]= xs[i][2]
        l = l+1
    else
        cat_2["x"][j]= xs[i][1]
        cat_2["y"][j]= xs[i][2]
        j=j+1
    end
end

In [7]:
x = {}
y = {}
for i=1,xs:size()[1] do
    x[i]= xs[i][1]
    y[i]= xs[i][2]
end

# Visualizing the given problem

In [8]:
-- scatter plots 
plot = Plot():circle(cat_1.x,cat_1.y, 'red', 'Category 1'):circle(cat_2.x, cat_2.y, 'blue', 'Category 2'):draw() 
plot:title('Data Points to separate'):redraw() 
plot:xaxis('random variable 1'):yaxis('random variable 2'):redraw() 
plot:legend(true) 
plot:save('out.html')

## Linear Models - Overview

### Linear Models : definition
A linear model is defined by the following equation :

$$ \hat{Y} = \hat{\beta_{0}} + \sum_{j=1}^{p}X_{j}*\hat{\beta_{j}}$$

With the $\beta$ vector being the model parameters.  This can be noted by the following vector product  : $$ \hat{Y} = \beta^T X$$ 

By design, linear models makes **huge** structural assumption about the data at hand. Thus sometimes, prediction power can be lowered but is very stable.

Such a discussion is about the **bias-variance dilemma** that is reminded at the end of this notebook.

### Fitting linear model

Linear models are usually fitted using the parameter combination that minimize the **Residual Sum of Square** _(RSS)_ parameter.

$$ RSS(\beta) = \sum_{i=1}^{N}(y_{i}-x^{T}_{i}\beta)^2$$

## Linear Regression for Classification using Torch

### Batch Gradient Descent

Batch Gradient Descent is an optimization method that is using several simple points of the data to optimize a given Loss function, as opposed to Stochastic Gradient Descent that only use single-sample points.

**Pro**

The benefit of batch gradient descent is that the trajectory of the weight vector is smoothed, compared to that in corresponding single-sample algorithms , since at each update the full set of misclassified patterns is used, the local statistical variations in the misclassified patterns tend to cancel while the large-scale trend does not.

Thus, if the samples are linearly separable, all of the possible correction vectors form a linearly separable set, and if η(k) satisfies. The sequence of weight vectors produced by the gradient descent algorithm for will always converge to a solution vector.

**Cons**

On the other side, if the algorithm is to be used in an online (and by online, we mean *REAL* online) manner, then fixed size batch can't be used since frequency of new samples arrival is to be determined.

It has to be reminded that Batch doesn't treat data points has a TimeSeries but in a iid manner.

For further analysis about Gradient Descent Techniques, please refer to [my notes on gradient descent procedures](https://www.dropbox.com/s/g3ovttvr3c02hb6/presentation_1.pdf?dl=0)

In [9]:
-- 2 : Creation of the model
model = nn.Linear(2,1)
criterion = nn.MSECriterion()
model:zeroGradParameters() -- Here, we are setting the cummulative of the gradients parameters to be equal to 0
model:reset() -- I don't know what this command is for but it was in the TP TODO : Look for it

Separating into train and test, because this is the way to do it rigth :)

In [10]:
gnuplot.plot(xs)

In [11]:
-- 3 : Learning Loop
learning_rate= torch.random(1,10)/10000 -- We put a random weight parameter in this.
maxEpoch = 50
all_losses={}
-- Well, might as well define a method so that we don't have to type it all the time

function train_batch(training_set,learning_rate,maxEpoch)
    assert(training_set.data ~= nil, 'please provide correct structure for the training set (i.e. training_set.data ~= nil)')
    assert(training_set.data ~= nil, 'please provide correct structure for the training set (i.e. training_set.label ~= nil)')
    local batch_loss = {}    
    model:zeroGradParameters()
    for iteration=1,maxEpoch do
        loss=0
        output = model:forward(training_set.data)
        loss= criterion:forward(output,training_set.label)
        model:backward(training_set.data,criterion:backward(model.output,training_set.label))
        updated_parameters = model:updateParameters(learning_rate)
        batch_loss[iteration] = loss
    end
    return batch_loss
end

In [12]:
all_losses['batch'] = {}
all_losses['batch'] = train_batch(training_set,learning_rate,maxEpoch)
gnuplot.plot(torch.Tensor(all_losses['batch']))

[string "-- 3 : Learning Loop..."]:8: please provide correct structure for the training set (i.e. training_set.data ~= nil)
stack traceback:
	[C]: in function 'assert'
	[string "-- 3 : Learning Loop..."]:8: in function 'train_batch'
	[string "all_losses['batch'] = {}..."]:2: in main chunk
	[C]: in function 'xpcall'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/david/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/david/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010d128d10: 

## Plotting train and test error

In [13]:
-- Lazily creating train and test sets, because we are lazy.


-----------------------  TRAIN   -----------------------  -----------------------
train = {}
train.data = {}
train.label = {}
train.data = xs[{{1,800},{}}]
train.label = ys[{{1,800},{}}]

setmetatable(train,
    {__index = function(t, i)
                return {t.data[i], t.label[i]}
               end})

function train:size()
    return self.data:size()
end

-----------------------  TEST   -----------------------  -----------------------

test = {}
test.data = {}
test.label = {}
test.data = xs[{{801,xs:size()},{}}]
test.label = ys[{{801,ys:size()},{}}]

setmetatable(test,
    {__index = function(t, i)
                return {t.data[i], t.label[i]}
               end})

function test:size()
    return self.data:size()
end

In [14]:
all_losses['batch'] = train_batch(learning_rate,maxEpoch)
gnuplot.plot(torch.Tensor(all_losses['batch'])) 

[string "-- 3 : Learning Loop..."]:8: attempt to index local 'training_set' (a number value)
stack traceback:
	[string "-- 3 : Learning Loop..."]:8: in function 'train_batch'
	[string "all_losses['batch'] = train_batch(learning_ra..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/david/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/david/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010d128d10: 

### Stochastic Gradient Descent 

### Set all_losses 

In [15]:
--all_losses['stochastic'] = nil
all_losses['stochastic'] = {}
all_losses['stochastic'][1] = {}

In [16]:
all_losses['stochastic'][1][1] = 1
all_losses['stochastic'][1][2] = 100

### Stochastic Gradient Descent implementation

In [17]:
-- 3 : Learning Loop
learning_rate= torch.random(1,10)/10000 -- We put a random weight parameter in this.
maxEpoch = 50
all_losses={}
-- Well, might as well define a method so that we don't have to type it all the time

function train_stochastic(training_set,learning_rate,maxEpoch)
    assert(training_set.data ~= nil, 'please provide correct structure for the training set (i.e. training_set.data ~= nil)')
    assert(training_set.data ~= nil, 'please provide correct structure for the training set (i.e. training_set.label ~= nil)')
    local stochastic_losses = {}    
    model:zeroGradParameters()
    for iteration=1,maxEpoch do
        loss=0
        for i=1,training_set.data:size()[1] do 
            output = model:forward(training_set.data[i])
            loss= loss + criterion:forward(output,training_set.label[i])
            model:backward(training_set.data[i],criterion:backward(model.output,training_set.label[i]))
            updated_parameters = model:updateParameters(learning_rate)
        end
        stochastic_losses[iteration] = loss
    end
    return stochastic_losses
end

In [18]:
training_set = {}
training_set['data'] = xs
training_set['label'] = ys
all_losses['stochastic'] = train_stochastic(training_set,learning_rate,maxEpoch)

In [19]:
gnuplot.plot(torch.Tensor(all_losses['stochastic'])) 

#### Using the StochasticGradient trainer

In [20]:
-- Setting up the trainer
trainer = nn.StochasticGradient(model,criterion)

The __StochasticGradient__ trainer of the torch nn module uses the following parameters

*  _learningRate :_ This is the learning rate used during training. The update of the parameters will be  parameters = parameters - learningRate * parameters_gradient . Default value is  0.01 .
*  _learningRateDecay :_ The learning rate decay. If non-zero, the learning rate (note: the field learningRate will not change value) will be computed after each iteration (pass over the dataset) with:  current_learning_rate =learningRate / (1 + iteration * learningRateDecay). The learning rate decay is  a 
*  _maxIteration :_ The maximum number of iteration (passes over the dataset). Default is  25 .
*  _shuffleIndices :_ Boolean which says if the examples will be randomly sampled or not. Default is  true . If  false , the examples will be taken in the order of the dataset.
*  _hookExample : Out of our scope
*  _hookIteration :_ Out of our scope

In [21]:
trainer.learningRate = learning_rate
trainer.maxIteration = maxEpoch

#### Adapting the data to the trainer persepective

In order to be able to use the torch trainer paradigm, we will have to do some preparation on our data :

1. We have to actually implements a 
2. We also have to be able to 

#### Adapting the data to the trainer paradigm

The trainer need an \__index method; 

This method is actually a way for LUA to provide inheritence. We will first set a metatable telling which metatable to look at without setting parameters

In [22]:
trainset = {}
trainset.data = xs
trainset.label = ys

setmetatable(trainset,
    {__index = function(t, i)
                return {t.data[i], t.label[i]}
               end})

function trainset:size()
    return self.data:size()
end

In [23]:
-- We ensure that the needed __index method is correctly implemented
getmetatable(trainset)

# Loading up differents datasets

In [24]:
require 'svm'

{
  hingeloss : function: 0x0d89f978
  spadd : function: 0x0dd9f8b0
  ascread : function: 0x0e203840
  ascwrite : function: 0x0e203820
  spdot : function: 0x0dbe5f50
  dataset : function: 0x0e203860
  SvmAsgd : table: 0x0e202048
  squaredhingeloss : function: 0x0dd2e2a0
  SvmSgd : table: 0x0e201740
  bininfo : function: 0x0dd02a20
  logloss : function: 0x0dbd24b8
  binread : function: 0x0dd77f70
}


In [25]:
?ascread -- :( Wish lib developers commented their code more othen. What about?
getmetatable(ascread)  -- Ok, no info on those things

[string "help(ascread -- :( Wish lib developers commen..."]:2: ')' expected (to close '(' at line 1) near 'getmetatable': 

In [26]:
d = svm.ascread('/Users/david/Documents/Master/M2/AS/notebooks/news20.bz2')
news_ = svm.binread('/Users/david/Documents/Master/M2/AS/notebooks/news20.bz2')


Reading /Users/david/Documents/Master/M2/AS/notebooks/news20.bz2	


/Users/david/torch/install/share/lua/5.1/svm/data.lua:65: bad argument #1 to 'lines' (/Users/david/Documents/Master/M2/AS/notebooks/news20.bz2: No such file or directory)
stack traceback:
	[C]: in function 'lines'
	/Users/david/torch/install/share/lua/5.1/svm/data.lua:65: in function 'ascread'
	[string "d = svm.ascread('/Users/david/Documents/Maste..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/david/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/david/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):

# Conclusion

Throughout this TME, we have gone through :

1. Linear problem statement
2. Visualization of a data separation problem
3. Prepare a dataset for classification purposes
4. Linear Models theory
5. Gradient Descent methods and good practices.
6. Loading data into Torch

Next week will be a totally new week.